In [1]:
#.env에서 API키 가져오기
import os
from dotenv import load_dotenv
load_dotenv('.env')
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')


In [4]:
import requests
import time
import json
import os
import re

CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

query_list = ["소설", "미스터리소설", "로맨스소설", "고전소설", "한국문학", "일본문학", "영미문학", "추리소설", "공포소설", "판타지소설", "역사소설", "무협소설", "라이트노벨"]

def search_books(query, start=1, display=100):
    url = "https://openapi.naver.com/v1/search/book.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,
        "start": start,
        "sort": "sim"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()["items"]
    else:
        print(f"⚠️ API 요청 실패 ({response.status_code}): {query}, start={start}")
        return []

all_books = []

for query in query_list:
    for start in range(1, 1000, 100):
        books = search_books(query, start)
        if not books:
            break
        all_books.extend(books)
        time.sleep(0.5)

unique_books = {}

for book in all_books:
    original_title = book["title"]
    title = normalize_title(original_title)  # ✅ 제목 정리 적용 (괄호 삭제, 숫자 정리)

    if exclude_edition_or_set(original_title):  # ✅ 원본 제목에서 "에디션"이 있는지 체크 (정리 전 체크)
        print(f"❌ 제외됨: {original_title}")  # 디버깅 출력 (배포 시 제거 가능)
        continue  

    if title not in unique_books:
        unique_books[title] = book

output_file = "filtered_novels.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(unique_books.values()), f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(unique_books)}권의 장편 소설 데이터가 '{output_file}'에 저장되었습니다!")


❌ 제외됨: 한강 스페셜 에디션 (작별하지 않는다, 흰, 검은 사슴 & 필사노트)
❌ 제외됨: 퇴마록 소장판 전권 세트 - 전17권
❌ 제외됨: 불편한 편의점 (김호연 장편소설, 40만부 기념 벚꽃 에디션)
❌ 제외됨: 데미안: 오리지널 초판본 표지디자인 (최신 원전 완역본)
❌ 제외됨: 어린 왕자(오리지널 초판본)(무선)
❌ 제외됨: 달러구트 꿈 백화점 (200만부 기념 합본호 : 아메리칸드림 에디션)
❌ 제외됨: 기생충 각본집 & 스토리보드북 세트
❌ 제외됨: 삼체 세트 (전3권ㅣ넷플릭스 드라마 '삼체' 원작)
❌ 제외됨: 미키 7 소설 세트 (봉준호 감독 영화 '미키 17' 원작)
❌ 제외됨: 세상의 마지막 기차역 (리커버 에디션)
❌ 제외됨: 지구 끝의 온실 (여름 에디션,김초엽 장편소설)
❌ 제외됨: 스토너(초판본)
❌ 제외됨: 스텔라 오디세이 트릴로지 세트 (김보영 장편소설)
❌ 제외됨: 노인과 바다(초판본)(1952년 오리지널 초판본 표지디자인) (1952년 오리지널 초판본 표지디자인)
❌ 제외됨: 싯다르타(초판본)(1922년 오리지널 초판본 표지디자인) (1922년 오리지널 초판본 표지디자인)
❌ 제외됨: 보그나르 주식회사 (김동식 AI 초단편선)
❌ 제외됨: 이방인(초판본)(1944년 오리지널 초판본 표지디자인) (1944년 오리지널 초판본 표지디자인)
❌ 제외됨: 변신(초판본)(1916년 오리지널 초판본 표지디자인) (1916년 오리지널 초판본 표지디자인)
❌ 제외됨: J.R.R. 톨킨 동화 선집 세트
❌ 제외됨: 아케치: D언덕 살인사건 (에도가와 란포 단편집)
❌ 제외됨: 아Q정전 · 광인일기 (루쉰 중단편선)
❌ 제외됨: 신기한 구름 + 잃어버린 옆모습 세트
❌ 제외됨: 체호프 단편선(완역본)
❌ 제외됨: 여자 없는 남자들 (헤밍웨이 단편선)
❌ 제외됨: 초판본 로미오와 줄리엣: 1597년 오리지널 초판본 표지디자인 (1597년 오리지널 초판본 표지디자인)
❌ 제외됨: 초판본 거울 나라의 앨리스: 1871년 오리지널 초판본 표지디자인 (

In [ ]:
import json
import random

# 📌 1. 전체 1637권 데이터 로드
with open("filtered_novels.json", "r", encoding="utf-8") as f:
    all_books = json.load(f)

# ✅ 2. 랜덤으로 200권 선택 (라벨링용)
random.shuffle(all_books)  # 랜덤 셔플
labeled_books = all_books[:200]  # 앞에서 200권 선택

# ✅ 3. 나머지 1437권 (모델 자동 요약용)
unlabeled_books = all_books[200:]

# 📌 4. 라벨링 데이터 저장
with open("labeling_novels.json", "w", encoding="utf-8") as f:
    json.dump(labeled_books, f, ensure_ascii=False, indent=4)

# 📌 5. 모델 요약용 데이터 저장
with open("unlabeled_books.json", "w", encoding="utf-8") as f:
    json.dump(unlabeled_books, f, ensure_ascii=False, indent=4)

print(f"✅ 랜덤 200권 라벨링 완료! ('labeling_novels.json' 저장)")
print(f"✅ 나머지 1437권 모델 요약 준비 완료! ('unlabeled_books.json' 저장)")


✅ 랜덤 200권 라벨링 완료! ('labeling_novels.json' 저장)
✅ 나머지 1481권 모델 요약 준비 완료! ('unlabeled_books.json' 저장)


In [ ]:
import json
import random

# 📌 1. 추가 라벨링 위해 1481권 데이터 로드
with open("unlabeled_books.json", "r", encoding="utf-8") as f:
    all_books = json.load(f)

# ✅ 2. 랜덤으로 100권 선택 (라벨링용)
random.shuffle(all_books)  # 랜덤 셔플
labeled_books = all_books[:100]  # 앞에서 200권 선택

# ✅ 3. 나머지 1381권 (모델 자동 요약용)
unlabeled_books = all_books[100:]

# 📌 4. 라벨링 데이터 저장
with open("labeling_novels_2.json", "w", encoding="utf-8") as f:
    json.dump(labeled_books, f, ensure_ascii=False, indent=4)

# 📌 5. 모델 요약용 데이터 저장
with open("unlabeled_books_final.json", "w", encoding="utf-8") as f:
    json.dump(unlabeled_books, f, ensure_ascii=False, indent=4)

print(f"✅ 랜덤 100권 라벨링 완료! ('labeling_novels_2.json' 저장)")
print(f"✅ 나머지 1337권 모델 요약 준비 완료! ('unlabeled_books_final.json' 저장)")
# labeling_novels_2.json + labeling_novels.json 합치고 unlabeled_books_2.json에서는 summary 제외해야함

✅ 랜덤 100권 라벨링 완료! ('labeling_novels_2.json' 저장)
✅ 나머지 1381권 모델 요약 준비 완료! ('unlabeled_books_final.json' 저장)


In [5]:
#summary 빼고 저장
import json

# ✅ 1️⃣ 기존 JSON 파일 로드
input_file = "unlabeled_books_final.json"
with open(input_file, "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 2️⃣ 모든 책에서 `summary` 필드 제거
for book in books_data:
    book.pop("summary", None)  # 🔹 `summary` 필드가 있으면 삭제

# ✅ 3️⃣ 새로운 JSON 파일로 저장
output_file = "unlabeled_books_no_summary.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(books_data, f, ensure_ascii=False, indent=4)

print(f"✅ `summary` 필드 제거 완료! 결과가 '{output_file}' 파일에 저장되었습니다.")


✅ `summary` 필드 제거 완료! 결과가 'unlabeled_books_no_summary.json' 파일에 저장되었습니다.


In [6]:
## 라벨링 데이터 중 "summary": "줄거리 없음"인 데이터 제거
import json

# ✅ 1️⃣ 기존 라벨링 데이터 로드
with open("labeling_novels.json", "r", encoding="utf-8") as f:
    labeled_books = json.load(f)

# ✅ 2️⃣ "줄거리 없음" 데이터 제거
filtered_books = [book for book in labeled_books if book["summary"].strip() != "줄거리 없음"]

# ✅ 3️⃣ 새로운 JSON 파일로 저장
output_file = "labeling_novels_filtered.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_books, f, ensure_ascii=False, indent=4)

print(f"✅ '줄거리 없음' 데이터를 제거하고, 결과를 '{output_file}' 파일에 저장했습니다!")


✅ '줄거리 없음' 데이터를 제거하고, 결과를 'labeling_novels_filtered.json' 파일에 저장했습니다!
